In [1]:
import numpy as np
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)
# on divise avec un train set et un test set
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ditmann\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def rejoin_words(tokenized_column):
    return ( " ".join(tokenized_column))

def tokenize_df(df, categorie):
    #on met tout en minuscule
    df[categorie] = df[categorie].str.lower()
    #on remplace les lignes de titre vides par NaN et on les supprime
    df['title'].replace('', np.nan, inplace=True)
    df.dropna(subset=[categorie], inplace=True)
    #suppression des stop words
    clean_title = df[categorie].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    #suppression de la ponctuation
    clean_title = clean_title.str.replace('[^\w\s]','')
    #tokenization
    tokenized_title = clean_title.apply(word_tokenize)
    #stemming
    stemmer = SnowballStemmer("english")
    stemmed_title = tokenized_title.apply(lambda x: [stemmer.stem(y) for y in x])
    #dé-tokenization
    final_title = stemmed_title.apply(lambda x: rejoin_words(x))
    #on remplace la colonne des titres par les nouveaux titres nettoyés
    df[categorie] = final_title

In [3]:
#traitement du set de train
data_train = pd.read_csv("bso-latest.csv", sep=",")
data_train = data_train.fillna("")
mask = (data_train['lang'] == 'en') 
data_train = data_train.loc[mask]
data_sample = data_train.sample(n=50000)
tokenize_df(data_sample, 'title')
tokenize_df(data_sample, 'publisher')
tokenize_df(data_sample, 'journal_name')
data_train = data_sample
    
#traitement du set de test
data_valid = pd.read_csv("https://raw.githubusercontent.com/azur-scd/datasets/main/barometre_oa/publis_scopus/20210914/doi_uniques_oa_attributs.csv",sep=",",encoding="utf-8")
data_valid = data_valid.fillna("")
data_valid.rename(columns = {'publisher':'old_publisher', 'publisher_by_doiprefix':'publisher'}, inplace = True)
tokenize_df(data_valid, 'title')
tokenize_df(data_valid, 'publisher')
tokenize_df(data_valid, 'journal_name')

data_train.reset_index(inplace = True)
data_valid.reset_index(inplace = True)
N_train = data_train.shape[0]
N_valid = data_valid.shape[0]
print("Train set : %d elements, Validation set : %d elements" %
                                               (N_train, N_valid))

C:\Users\ditmann\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11,12,16,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-2-4057dc6d5ed0>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_title = clean_title.str.replace('[^\w\s]','')
<ipython-input-2-4057dc6d5ed0>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_title = clean_title.str.replace('[^\w\s]','')
<ipython-input-2-4057dc6d5ed0>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_title = clean_title.str.replace('[^\w\s]','')
<ipython-input-2-4057dc6d5ed0>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_title = clean_title.str.replace('[^\w\s]','')


Train set : 50000 elements, Validation set : 13597 elements


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from spacy.lang.en import English

nlp = English()

# Vectorization parameters
# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 2)
# Limit on the number of features. We use the top 20K features.
TOP_K = 20000
# Whether text should be split into word or character n-grams.
# One of 'word', 'char'.
TOKEN_MODE = 'word'
# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

def ngram_vectorize(train_texts, train_labels, val_texts):  
    
    """Vectorizes texts as n-gram vectors.1 text = 1 tf-idf vector the length of vocabulary of unigrams + bigrams.
  # Arguments
      train_texts: list, training text strings.
      train_labels: np.ndarray, training labels.
      val_texts: list, validation text strings.
   # Returns 
       x_train, x_val: vectorized training and validation texts
    """
  # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
      'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
      'dtype': 'int32',
      'strip_accents': 'unicode',
      'decode_error': 'replace', 
      'analyzer': TOKEN_MODE,  # Split text into word tokens.
       'min_df': MIN_DOCUMENT_FREQUENCY,
       }
    vectorizer = TfidfVectorizer(**kwargs)
  # Learn vocabulary from training texts and vectorize training texts.
    x_train = vectorizer.fit_transform(train_texts)  # Vectorize validation texts.
    x_val = vectorizer.transform(val_texts)  # Select top 'k' of the vectorized features.
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val


In [5]:
data_train_final = data_train['title'] + ' ' + data_train['publisher'] + ' ' + data_train['journal_name']
data_valid_final = data_valid['title'] + ' ' + data_valid['publisher'] + ' ' + data_valid['journal_name']

from sklearn import preprocessing

#encodage de la valeur cible
le = preprocessing.LabelEncoder()
data_train["bso_classification"] = le.fit_transform(data_train["bso_classification"])

x_train,x_val = ngram_vectorize(data_train_final, data_train["bso_classification"], data_valid_final)

C:\Users\ditmann\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1805: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn("Only {} 'dtype' should be used. {} 'dtype' will "


In [6]:
y_train = data_train["bso_classification"]

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
x_smote, y_smote = sm.fit_resample(x_train, y_train)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

RandFores_clf =RandomForestClassifier( n_estimators=50
                            ,class_weight="balanced",n_jobs=-1)
RandFores_clf.fit(x_smote, y_smote)
domain = RandFores_clf.predict(x_val)

In [13]:
csv_final = pd.read_csv("https://raw.githubusercontent.com/azur-scd/datasets/main/barometre_oa/publis_scopus/20210914/doi_uniques_oa_attributs.csv",sep=",",encoding="utf-8")
csv_final['title'].replace('', np.nan, inplace=True)
csv_final.dropna(subset=['title'], inplace=True)
csv_final['domain'] = le.inverse_transform(domain)

In [17]:
csv_final.to_csv('publications_index.csv')

In [18]:
csv_final

,source_id,doi,year,corresponding,all_authors,genre,title,published_date,year_upw,publisher,...,dsm_policy_published,dsm_romeo_id,doi_prefix,publisher_by_doiprefix,published-online-date,published-print-date,journal-published-print-date,is-referenced-by-count,funder,domain
0,SCOPUS_ID:84889561052,10.1111/ger.12094,2016,Lassauzay C.|UCA (affiliation),Lassauzay C.,journal-article,Assessment of the oral health knowledge of hea...,2013-12-08,2013.0,Wiley,...,cannot,2050.0,10.1111,Wiley,2013.0,2016.0,2016.0,6.0,NaN,Medical research
1,SCOPUS_ID:84908431549,10.1016/j.lrp.2014.09.001,2016,NaN,Saglietto L.|Loufrani-Fedida S.,journal-article,Mechanisms for Managing Competencies in Projec...,2016-02-01,2016.0,Elsevier BV,...,cannot,30.0,10.1016,Elsevier BV,NaN,2016.0,2016.0,17.0,NaN,Medical research
2,SCOPUS_ID:84909580664,10.1007/978-1-349-26270-0,2016,NaN,Gaffard J.L.,book,New Theories in Growth and Development,1998-01-01,1998.0,Palgrave Macmillan UK,...,NaN,NaN,10.1007,Springer Science and Business Media LLC,NaN,1998.0,NaN,4.0,NaN,Mathematics
3,SCOPUS_ID:84931086752,10.1007/s10750-015-2333-y,2016,NaN,Thibaut T.,journal-article,The necromass of the Posidonia oceanica seagra...,2015-06-17,2015.0,Springer Science and Business Media LLC,...,cannot,74.0,10.1007,Springer Science and Business Media LLC,2015.0,2016.0,2016.0,40.0,NaN,Biology (fond.)
4,SCOPUS_ID:84940831274,10.1136/annrheumdis-2015-207336,2016,NaN,Leroy S.,journal-article,A prospective study of the 6 min walk test as ...,2015-08-31,2015.0,BMJ,...,cannot,25.0,10.1136,BMJ,2015.0,2016.0,2016.0,11.0,NaN,Medical research
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13594,SCOPUS_ID:85114491803,10.3390/e23091156,2021,NaN,Laure P.,journal-article,Octree Optimized Micrometric Fibrous Microstru...,2021-09-02,2021.0,MDPI AG,...,can,487.0,10.3390,NaN,2021.0,NaN,NaN,0.0,NaN,"Physical sciences, Astronomy"
13595,SCOPUS_ID:85114506994,10.1111/1744-7917.12937,2021,NaN,Moussian B.,journal-article,Chitin synthase 1 and five cuticle protein gen...,2021-09-09,2021.0,Wiley,...,cannot,2050.0,10.1111,Wiley,2021.0,NaN,NaN,0.0,NaN,Biology (fond.)
13596,SCOPUS_ID:85114520470,10.1007/s00367-021-00713-8,2021,NaN,Michaud F.,journal-article,Sedimentology and structure of a Holocene slum...,2021-09-08,2021.0,Springer Science and Business Media LLC,...,cannot,3298.0,10.1007,Springer Science and Business Media LLC,2021.0,2021.0,2021.0,0.0,Dirección General de Asuntos del Personal Acad...,Biology (fond.)
13597,SCOPUS_ID:85114553710,10.1109/ICDL49984.2021.9515660,2021,NaN,Romero M.|Roux L.,proceedings-article,Formalizing Problem Solving in Computational T...,2021-08-23,2021.0,IEEE,...,cannot,38,10.1109,Institute of Electrical and Electronics Engine...,NaN,2021.0,NaN,1.0,NaN,Computer and \n information sciences
